In [ ]:
import sys
import os


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

RESULTS_PATH = 'results.csv'
OPTUNA_DATABASE = 'study_results.db'
BASE_CONFIGURATION_NAME = 'configuration'


In [ ]:
from automl.loggers.result_logger import ResultLogger
import optuna
import optuna.visualization as vis
from automl.utils.optuna_utils import load_study_from_database
import matplotlib.pyplot as plt


# Load the experiment

In [ ]:
base_experiment_path = "C:\\Experiments\\rl-zoo-CartPole-dqn-2\\HPOptimizationExperiments\\2\\experiments"
experiment_relative_path = 'original'
experiment_path = f'{base_experiment_path}\\{experiment_relative_path}'

In [ ]:
if not os.path.exists(experiment_path):
    raise Exception("DOES NOT EXIST")

# Evaluation of HyperparameterOptimizationPipeline

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_results_logger

hyperparameter_optimization_results = get_hp_opt_results_logger(experiment_path)

print(f"Hyperparameter_optimization_results in path: {hyperparameter_optimization_results.get_artifact_directory()}")

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_hp_opt_optuna_study


optuna_study = get_hp_opt_optuna_study(hyperparameter_optimization_results)


## Hyperparameter Study

In [ ]:
try:
    print(f"optuna_study done with with best value {optuna_study.best_value} in trial {optuna_study.best_trial.number} with best parameters:\n{optuna_study.best_params}")

except:
    print("No best trial yet")

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import print_optuna_trials_info

print_optuna_trials_info(optuna_study)

In [ ]:
from optuna.visualization import plot_slice

fig = plot_slice(optuna_study)
fig.show()

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import print_optuna_param_importances

print_optuna_param_importances(optuna_study)

In [ ]:
fig = vis.plot_param_importances(optuna_study)
fig.show()

In [ ]:


fig = vis.plot_parallel_coordinate(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_intermediate_values(optuna_study)
fig.show()

In [ ]:
fig = vis.plot_optimization_history(optuna_study)
fig.show()

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import plot_scattered_values_for_param

if False: # this is not really necessary here, it is best to use it studying single configurations

    try:
        plot_scattered_values_for_param(optuna_study)

    except Exception as e:
        print(f"Could not plot scattered values because of error: {e}")

In [ ]:
parameters_to_plot = []
# parameters_to_plot = ["hidden_size", "hidden_layers"]

if len(parameters_to_plot) > 1:

    fig = vis.plot_contour(optuna_study, params=parameters_to_plot)
    fig.show()

# Global evaluation of configurations

In [ ]:
AGGREGATE_NUMBER = 10 #the number of neighbor points to sum to plot the needed graphs

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import study_of_configuration                          


In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_results_of_configurations                          



results_of_configurations : dict[str, ResultLogger] = get_results_of_configurations(experiment_path)

In [ ]:
print(f"Configurations:  {results_of_configurations.keys()}")

# Global view of performance

In [ ]:
steps_done = hyperparameter_optimization_results.list_of_unique_values("step")
print(steps_done)

In [ ]:
for step in steps_done:

    hyperparameter_optimization_results.plot_bar_graph(x_axis='experiment', y_axis='result', to_show=False, fixed_value_tuple=("step", step))
    hyperparameter_optimization_results.plot_linear_regression(x_axis='experiment', y_axis='result', to_show=False, fixed_value_tuple=("step", step))
    hyperparameter_optimization_results.plot_current_graph(title=f'experiments_at_step_{step}')

## Pruned Performances

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_pruned_trials                          


pruned_optuna_trials, pruned_optuna_trials_per_steps, pruned_trials = get_pruned_trials(optuna_study)

print(f"Pruned trials: {pruned_trials}")


### Worst Pruned Trials

In [ ]:
WORST_PRUNED = 3

In [ ]:
for n_completed_steps in pruned_optuna_trials_per_steps.keys():
    
    pruned_optuna_trials : list[optuna.Trial] = pruned_optuna_trials_per_steps[n_completed_steps][0:WORST_PRUNED] 
    
    pruned_trials = [f'configuration_{trial.number}' for trial in pruned_optuna_trials ]

    for configuration_name in pruned_trials:
        
        results_logger = results_of_configurations[configuration_name]
    
        study_of_configuration(configuration_name, results_logger)

### Best Pruned Trials

In [ ]:
BEST_PRUNED = 3

In [ ]:
for n_completed_steps in pruned_optuna_trials_per_steps.keys():
    
    #correct this
    pruned_optuna_trials : list[optuna.Trial] = pruned_optuna_trials_per_steps[n_completed_steps][-BEST_PRUNED:] 
    
    pruned_trials = [f'configuration_{trial.number}' for trial in pruned_optuna_trials ]

    for configuration_name in pruned_trials:
        
        results_logger = results_of_configurations[configuration_name]
    
        study_of_configuration(configuration_name, results_logger)

## Completed Trials Evaluation

In [ ]:
completed_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.COMPLETE]

completed_optuna_trials.sort(key=lambda trial: trial.value) # sort given the trial value


### Worst Completed Performances

In [ ]:
N_WORST = 3

In [ ]:

worst_optuna_trials = completed_optuna_trials[0:N_WORST]

worst_configurations_to_study = [f"configuration_{trial.number}" for trial in worst_optuna_trials]

print(f"Worst configurations to study: {worst_configurations_to_study}")

In [ ]:
for configuration_name in worst_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    study_of_configuration(configuration_name, results_logger)



## Non Completed Performances

In [ ]:
non_completed_optuna_trials = [trial for trial in optuna_study.trials if trial.state == optuna.trial.TrialState.RUNNING or trial.state == optuna.trial.TrialState.WAITING]

non_completed_optuna_trials.sort(key=lambda trial: trial.value) # sort given the trial value

### All non completed trials

In [ ]:
non_completed_trials = [f"configuration_{trial.number}" for trial in non_completed_optuna_trials]


In [ ]:
for configuration_name in non_completed_trials:

    try:
    
        results_logger = results_of_configurations[configuration_name]

        study_of_configuration(configuration_name, results_logger)

    except:
        print(f"Could not look into non completed configuration: {configuration_name}")

## Best Performances

In [ ]:
N_BEST = 3

In [ ]:
best_optuna_trials = completed_optuna_trials[(len(completed_optuna_trials) - N_BEST):]

best_configurations_to_study = [f"configuration_{trial.number}" for trial in best_optuna_trials]

print(f"Best configurations to study: {best_configurations_to_study}")

In [ ]:
for configuration_name in best_configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]

    study_of_configuration(configuration_name, results_logger)

# Smaller study

## Configuration study in optuna

In [ ]:
#trials_to_study = [0]


In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import get_trials_with_decreasing_intermediates, print_intermidiate_values
trials_to_study = get_trials_with_decreasing_intermediates(optuna_study)


In [ ]:

print_intermidiate_values(trials_to_study, optuna_study)

In [ ]:
from automl.hp_opt.hp_eval_results.hp_eval_results import plot_scattered_values_for_param

try:
    plot_scattered_values_for_param(optuna_study, trials_to_study)

except Exception as e:
    print(f"Could not plot scattered values because of error: {e}")

## Results study

In [ ]:
configurations_to_study = [f'configuration_{trial_n}' for trial_n in trials_to_study]

In [ ]:

try:

    for configuration_name in configurations_to_study:
    
        results_logger = results_of_configurations[configuration_name]

        study_of_configuration(configuration_name, results_logger)

except KeyError as e:
    print(f"KeyError: {e}\nAvailable keys are {results_of_configurations.keys()}")
    raise e

### Study for each agent

In [ ]:
agents_in_study = []
# agents_in_study = ["agent_1", "agent_2"]

In [ ]:
agents_to_study : dict[str, ResultLogger]= {}

for configuration_name in configurations_to_study:
    
    results_logger = results_of_configurations[configuration_name]
    
    for agent_name in agents_in_study:
      
        agent_results_logger = ResultLogger(input={
                                            "logger_directory" : f"{results_logger.lg.logDir}\\{agent_name}",
                                            "filename" : RESULTS_PATH,
                                            "create_new_directory" : False
                                          })

        agents_to_study[f"{configuration_name}_{agent_name}"] = agent_results_logger
        
        agent_results_logger.proccess_input()


In [ ]:
for agent_name, agent_results_logger in agents_to_study.items():
    
    study_of_configuration(agent_name, agent_results_logger)